In [1]:
#!python3
import pandas as pd
file = "data/gurume.csv"

#### pd.read_csv(ファイル名, encoding)
- data/gurume.csvを読み込む
- encodingをWindows標準のcp932に指定する（しないと読み込みエラーになる）
- 読み込んだ結果はデータフレームdfとして返される。
- df[列名]で列アクセス、i行目が欲しい場合はdf.iloc[i]とする
- df.columnsで列の一覧

In [6]:
df = pd.read_csv(file, encoding="cp932")

In [26]:
df.shape

(10507, 53)

#### xmlから緯度経度情報を取得する
標準のxmlモジュールを調べてがんばる

In [333]:
import xml.etree.ElementTree as ET
tree = ET.parse('data/map.osm')
root = tree.getroot()

In [335]:
root[0].attrib


{'minlat': '35.6384000',
 'minlon': '139.6947000',
 'maxlat': '35.6480000',
 'maxlon': '139.7083000'}

In [336]:
d = root[0].attrib
minlat = float(d["minlat"])
minlon = float(d["minlon"])
maxlat = float(d["maxlat"])
maxlon = float(d["maxlon"])

#### 範囲内の緯度経度取得
- df["lat"] > minlat でbool配列が得られる
- bool配列同士を &, | などで演算できる
- df[bool配列] でTrueの行が取得できる

In [337]:
# 範囲内boolインデックス
b1 = df["lng"] > minlon
b2 = df["lng"] < maxlon
b3 = df["lat"] < maxlat
b4 = df["lat"] > minlat
ind = (b1 & b2) & (b3 & b4)
ind.sum()

97

In [338]:
# boolがtrueの列を取得
shops = df[ind]

#### 四分位点の計算
- df[列名].quantile([q1, q2, q3,...])  
で下から割合q1の値、q2の値、…が求まる

In [339]:
qs = shops["score"].quantile([0.25, 0.75])
high = qs[0.75]
low = qs[0.25]
print(high, low)

3.43 3.05


#### スコアに応じた表示色の設定
第一四分位点以下を青に、第三四分位点以上を赤に設定する

In [340]:
shops["color"] = "green"

/Users/owner/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [341]:
ind = shops.score > high
shops["color"][ind] = "red"


/Users/owner/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [342]:
ind2 = shops.score < low
shops["color"][ind2] = "blue"

/Users/owner/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [343]:
# shops[["score","color"]][shops["color"]=="blue"]
shops[["score","color"]]

,score,color
497,3.18,green
537,3.44,red
599,3.07,green
871,3.00,blue
945,3.31,green
982,3.19,green
1009,3.02,blue
1380,3.06,green
1411,3.03,blue
1476,3.08,green


In [295]:
shops.head()

,name,access,address,band,barrier_free,budget,budget_memo,capacity,card,catch,...,show,small_area,station_name,sub_genre,tatami,tv,urls,wedding,wifi,color
497,なゝ樹 恵比寿本店,東京メトロ日比谷線【恵比寿駅】徒歩２分ＪＲ山手線他【恵比寿駅】西口 徒歩3分東急東横線【代官...,東京都渋谷区恵比寿西１-13-2 サンキビル 1F,不可,なし,"OrderedDict([('code', 'B003'), ('name', '3001～...",NaN,50.0,利用可,【完全個室】宴会に◎ 鶏焼き 390円～≪地鶏≫,...,なし,"OrderedDict([('code', 'X115'), ('name', '恵比寿')])",恵比寿,"OrderedDict([('code', 'G008'), ('name', '焼肉・ホル...",なし,なし,"OrderedDict([('pc', 'https://www.hotpepper.jp/...",NaN,あり,green
537,いえ村 恵比寿,JR恵比寿駅西口徒歩3分。ファミリーマートさんを過ぎてすぐに左折し、JACK POTさんの隣...,東京都渋谷区恵比寿西１-13-19,不可,なし ：無,"OrderedDict([('code', 'B004'), ('name', '5001～...",有:500円※コース注文の際はチャージ料込,50.0,利用可,記念日やお誕生日会に♪ 美味い料理と旨い酒,...,なし,"OrderedDict([('code', 'X115'), ('name', '恵比寿')])",恵比寿,"OrderedDict([('code', 'G004'), ('name', '和食')])",なし ：お座敷フロアのご用意はございませんが、お席のレイアウト等、お気軽にご相談下さい,あり,"OrderedDict([('pc', 'https://www.hotpepper.jp/...",貸切応相談 店舗まで一度御連絡の上、御相談ください。,あり,green
599,和食居酒屋 恵比寿 凛音,JR恵比寿駅西口 徒歩2分／地下鉄日比谷線恵比寿駅 徒歩2分 恵比寿駅から193m,東京都渋谷区恵比寿西１－８－１３ エムロード戸田３Ｆ,不可,なし ：事前にご相談頂ければ、ご来店のお手伝いをさせて頂きます。,"OrderedDict([('code', 'B003'), ('name', '3001～...",テーブルチャージ500円,40.0,利用可,貸切は25名～45名まで対応 厳選した全国各地の地酒,...,なし,"OrderedDict([('code', 'X115'), ('name', '恵比寿')])",恵比寿,"OrderedDict([('code', 'G004'), ('name', '和食')])",なし ：。★恵比寿 居酒屋 貸切 地鶏 鍋 肉 焼肉 日本酒,なし,"OrderedDict([('pc', 'https://www.hotpepper.jp/...",お気軽にご相談下さい,未確認,green
871,個室居酒屋 えびす丸 恵比寿丸 恵比寿駅前店,【JR恵比寿駅西口 徒歩1分】,東京都渋谷区恵比寿西１-１３-３ ROOB６ ２F,不可,なし ：◆えびす丸 もつ鍋 焼き鳥 個室居酒屋 忘年会 新年会,"OrderedDict([('code', 'B002'), ('name', '2001～...",サービス料10％・お通しあり（宴会コースは全て込の価格）,90.0,利用可,忘年会に最適な完全個室♪ 九州博多天神もつ鍋,...,あり,"OrderedDict([('code', 'X115'), ('name', '恵比寿')])",恵比寿,"OrderedDict([('code', 'G004'), ('name', '和食')])",あり ：お座敷個室あり（8名～30名様用）／掘りごたつ個室あり（2名～6名様用）,なし,"OrderedDict([('pc', 'https://www.hotpepper.jp/...",【歓迎】 ご予算・人数など、お気軽にご相談ください♪,未確認,blue
945,さかえや 恵比寿本店,JR恵比寿駅西口 徒歩2分,東京都渋谷区恵比寿西１－９－１２,不可,なし,"OrderedDict([('code', 'B003'), ('name', '3001～...",お通し代 450円（不要の場合はお申し出下さい）/サービス料5％,66.0,利用可,個室は2名様用～ご用意◎ 接待・デートにおすすめ,...,なし,"OrderedDict([('code', 'X115'), ('name', '恵比寿')])",恵比寿,"OrderedDict([('code', 'G001'), ('name', '居酒屋')])",あり,なし,"OrderedDict([('pc', 'https://www.hotpepper.jp/...",ご相談下さい！週末のご宴席歓迎です！お問合せ下さい。,あり,green


#### 文字列処理
文字列.split()

#### folium地図取得
さっき取得した緯度経度を使う

In [299]:
import folium

#### マーカーの配置
さっき取得した色情報やスコア、店名を使う

In [344]:
lat = (minlat + maxlat) / 2
lon = (minlon + maxlon) / 2
m = folium.Map(location=[lat, lon], zoom_start=14)

In [345]:
icon =  folium.Icon(color=shops["color"].iloc[0])
icon

In [302]:
tooltip = "genre: %s \n score: %s" % (shops["genre"].iloc[0], shops["score"].iloc[0])

In [226]:
marker = folium.Marker([shops["lat"].iloc[0], shops["lng"].iloc[0]],
                       icon=icon,
                       tooltip=shops["name"].iloc[0],
                       popup="genre: %s \n score: %s" % (shops["genre"].iloc[0], shops["score"].iloc[0]))
marker.add_to(m)
m

In [256]:
shops["genre"].iloc[0].split("'")[7]

'居酒屋'

In [347]:
# shops["genre_new"] = 
shops["genre_new"] = shops["genre"].str.split("'",expand=True)[7]

/Users/owner/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [348]:
m = folium.Map(location=[lat, lon], zoom_start=14)
for i in range(shops.shape[0]):
    icon =  folium.Icon(color=shops["color"].iloc[i])
    genre = shops["genre_new"].iloc[i]
    marker = folium.Marker([shops["lat"].iloc[i], shops["lng"].iloc[i]],
                       icon=icon,
                       tooltip=shops["name"].iloc[i],
                       popup="genre: %s <br> score: %s" % (genre, shops["score"].iloc[i]))
    marker.add_to(m)

folium.Marker([minlat, minlon]).add_to(m)
folium.Marker([maxlat, maxlon]).add_to(m)
m.save('data/rukamap.html')

,genre_new,genre
1119,居酒屋,"OrderedDict([('code', 'G001'), ('name', '居酒屋')..."
1181,居酒屋,"OrderedDict([('code', 'G001'), ('name', '居酒屋')..."
1399,居酒屋,"OrderedDict([('code', 'G001'), ('name', '居酒屋')..."
1599,居酒屋,"OrderedDict([('code', 'G001'), ('name', '居酒屋')..."
1746,焼肉・ホルモン,"OrderedDict([('code', 'G008'), ('name', '焼肉・ホル..."
1848,居酒屋,"OrderedDict([('code', 'G001'), ('name', '居酒屋')..."
1893,居酒屋,"OrderedDict([('code', 'G001'), ('name', '居酒屋')..."
2485,居酒屋,"OrderedDict([('code', 'G001'), ('name', '居酒屋')..."
2566,居酒屋,"OrderedDict([('code', 'G001'), ('name', '居酒屋')..."
2645,中華,"OrderedDict([('code', 'G007'), ('name', '中華'),..."
